# This is a program to test whether a controlled foreign company has a CFC charge

In [25]:
# import libraries
import csv
import pandas as pd

In [26]:
# create list of excluded territories from the excluded_territories csv file
excluded_territories_object = csv.reader(open('excluded_territories.csv', 'r'))
excluded_territories = []
for row in excluded_territories_object:
    excluded_territories.append(row[0].lower())

# create list of simplified territories from the simplified_territories csv file
simplified_territories_object = csv.reader(open('simplified_territories.csv', 'r'))
simplified_territories = []
for row in simplified_territories_object:
    simplified_territories.append(row[0].lower())

In [27]:
# create a self class
class Company():
    def __init__(self,
                 name = None,
                 country_residence = None,
                 currency = None,
                 
                 pbt = None,
                 div_inc_no_ded = None,
                 prof_prop = None,
                 gain_tang_ass = None,
                 gain_shares = None,
                 gain_ip = None,
                 gain_cap_other = None,
                 trust_inc = None,
                 part_inc = None,
                 tp_adj = None,

                 gross_int_exp = None,
                 op_exp = 0.001, # this is to avoid division by zero error
                 op_exp_interco = None,
                 goods_costs = None,

                 exempt_inc = None,
                 low_tax_inc = None,
                 tax_paid = None,
                 notional_int = None,
                 ruling_inc = None,
                 tp_adj_asym = None,

                 div_inc = None,
                 int_inc = None,
                 forex_gain_loans = None,
                 forex_gain_non_trade = None,
                 derivative_inc = None,
                 fin_lease_inc = None,
                 roy_inc = None,
                 non_trade_other = None):
        
        self.name = name
        self.country_residence = country_residence
        self.currency = currency
        
        # attributes for lpe test
        self.pbt = pbt
        self.div_inc_no_ded = div_inc_no_ded
        self.prof_prop = prof_prop
        self.gain_tang_ass = gain_tang_ass
        self.gain_shares = gain_shares
        self.gain_ip = gain_ip
        self.gain_cap_other = gain_cap_other
        self.trust_inc = trust_inc
        self.part_inc = part_inc
        self.tp_adj = tp_adj

        # attributes for lpme test
        self.gross_int_exp = gross_int_exp
        self.op_exp = op_exp
        self.op_exp_interco = op_exp_interco
        self.goods_costs = goods_costs

        #attributes for ete test            
        self.exempt_inc = exempt_inc
        self.low_tax_inc = low_tax_inc
        self.tax_paid = tax_paid
        self.notional_int = notional_int
        self.ruling_inc = ruling_inc
        self.tp_adj_asym = tp_adj_asym

        self.div_inc = div_inc
        self.int_inc = int_inc
        self.forex_gain_loans = forex_gain_loans
        self.forex_gain_non_trade = forex_gain_non_trade
        self.derivative_inc = derivative_inc
        self.fin_lease_inc = fin_lease_inc
        self.roy_inc = roy_inc
        self.non_trade_other = non_trade_other


    # simplified territory exemption (inside simplified territories): create a method to check if self residence is one of the simplified territories
    def st_exemption(self):
        if self.country_residence.lower() in simplified_territories:
            return True
        else:
            return False
    
    # low profit exemption test:
    # first calculate accounting profit
    def ap(self):
        if (self.pbt == None or
            self.div_inc_no_ded == None or
            self.prof_prop == None or
            self.gain_tang_ass == None or
            self.gain_shares == None or
            self.gain_ip == None or
            self.gain_cap_other == None or
            self.trust_inc == None or
            self.part_inc == None):
            return 'Missing values to determine ap'
        else:
            return self.pbt - self.div_inc_no_ded - self.prof_prop - self.gain_tang_ass - self.gain_shares - self.gain_ip - self.gain_cap_other + self.trust_inc + self.part_inc
        
    def lpe(self):
            if self.tp_adj == None:
                return 'Missing tp adjustment on intercompany loans to determine if lpe conditions are satisfied'
            elif type(self.ap()) == str:
                return self.ap()
            elif self.tp_adj < 50000 and self.ap() < 50000:
                return True
            elif self.tp_adj > 50000 and (self.ap() + self.tp_adj) < 50000:
                return True
            elif self.ap() < 500000 and self.tp_adj < 50000:
                if type(self.non_trade_inc()) == str:
                    return self.non_trade_inc()
                elif self.non_trade_inc() < 50000:
                    return True
                else:
                    return False
            else:
                return False

    # low profit margin exemption:
    # first calculate relevant operating expenditure
    def roo(self):
        if (self.op_exp == None or
            self.op_exp_interco == None
            or self.goods_costs == None):
            return 'Missing values to determine relevant operating expenditure'
        else:
            return self.op_exp - self.op_exp_interco - self.goods_costs
    
        
    def lpme(self):
        if type(self.ap()) == str:
            return self.ap()
        elif type(self.roo()) == str:
            return self.roo()
        elif self.gross_int_exp == None:
            return 'Missing gross interest to determine if lpme conditions are satisfied'
        elif (self.ap() + self.gross_int_exp) / self.roo() < 0.1:
            return True
        else:
            return False


    # excluded territories exemption:
    # first calculate relevant income
    def ri(self):
        if (self.exempt_inc == None or
            self.low_tax_inc == None or
            self.tax_paid == None or
            self.notional_int == None or
            self.trust_inc == None or
            self.part_inc == None or
            self.ruling_inc == None or
            self.tp_adj_asym == None):
            return 'Missing values to determine relevant income'
        else:
            return self.exempt_inc + self.low_tax_inc + self.tax_paid + self.notional_int + self.trust_inc + self.part_inc + self.ruling_inc + self.tp_adj_asym
    
    # secondly calculate non-trading income
    def non_trade_inc(self):
        if (self.div_inc == None or
            self.int_inc == None or
            self.forex_gain_loans == None or
            self.forex_gain_non_trade == None or
            self.derivative_inc == None or
            self.fin_lease_inc == None or
            self.roy_inc == None or
            self.non_trade_other == None):
            return 'Missing values to determine non-trading income'
        else:
            return self.div_inc + self.int_inc + self.forex_gain_loans + self.forex_gain_non_trade + self.derivative_inc + self.fin_lease_inc + self.roy_inc + self.non_trade_other
        
    def ete(self):
        if self.country_residence not in excluded_territories:
            return False
        elif type(self.ap()) == str:
            return self.ap()
        elif type(self.ri()) == str:
            return self.ri()
        elif (self.ri() / self.ap() < 0.1):
            return True
        elif type(self.non_trade_inc()) == str:
            return self.non_trade_inc()
        elif self.non_trade_inc() < 50000:
            return True
        else:
            return False

In [28]:
# list of headings for pandas dataframe

headers = ['name',
           'country_residence',
           'currency',
                
           'pbt',
           'prof_prop',
           'gain_tang_ass',
           'gain_shares',
           'gain_ip',
           'gain_cap_other',
           'trust_inc',
           'part_inc',
           'tp_adj',

           'gross_int_exp',
           'op_exp',
           'op_exp_interco',
           'goods_costs',
              
           'exempt_inc',
           'low_tax_inc',
           'tax_paid',
           'notional_int',
           'ruling_inc',
           'tp_adj_asym',
              
           'div_inc',
           'int_inc',
           'forex_gain_loans',
           'forex_gain_non_trade',
           'derivative_inc',
           'fin_lease_inc',
           'roy_inc',
           'non_trade_other']


In [29]:
dir(Company)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'ap',
 'ete',
 'lpe',
 'lpme',
 'non_trade_inc',
 'ri',
 'roo',
 'st_exemption']

In [30]:
# Function to create a single company instance and add it to the list
# i.e. a list of company objects
company_instances = []

def create_company_instance():
    # Get the company name from the user
    name = input("Enter the company name: ")
    # Get the company residence from the user
    country_residence = input("Enter the company residence: ")
    # Get the currency from the user
    currency = input("Enter the currency: ")
    # Create an instance of the class
    company = Company(name, country_residence, currency)
    # Add the instance to the list
    company_instances.append(company)


In [31]:
for company in company_instances:
    print(company.__dict__)

In [32]:
# Create a function to run the exemption tests in order

def run_exemption_tests(cfc):
    print('Simplified territory exemption: ' + str(cfc.st_exemption()),
    '\n\nLow profit exemption: ' + str(cfc.lpe()),
    '\n\nLow profit margin exemption: ' + str(cfc.lpme()),
    '\n\nExcluded territory exemption: ' + str(cfc.ete()) + '\n\n')
    
    if cfc.st_exemption() == True or cfc.lpe() == True or cfc.lpme() == True or cfc.ete() == True:
        print('CFC is exempt from CFC rules')
        return True
    elif cfc.st_exemption() == False and cfc.lpe() == False and cfc.lpme() == False and cfc.ete() == False:
        print('CFC is not exempt from CFC rules')
        return False
    else:
        print('Missing information to complete exemption tests')
        return False

In [42]:
# create a function that will create a company and run the tests as attributes are entered

def com_exp_tst():
    global cfc
    cfc = Company()
    cfc.name = input("Enter the company name: ")
    cfc.country_residence = input("Enter the country of residence: ")

    if cfc.st_exemption():
        print("The company is resident in a simplified territory and is exempt from the CFC charge")
        return True
    else:
        cfc.pbt = int(input("Company's profit/(loss) before tax? "))
        cfc.div_inc_no_ded = int(input("Dividends or other distributions received from other companies, except for any dividend/distribution where a tax deduction is allowed in any territory in respect of the payment? "))
        cfc.prof_prop = int(input("Profit/(loss) derived from the renting of property? "))
        cfc.gain_tang_ass = int(input("Gain/(loss) on the disposal or revaluation/impairment of tangible fixed assets? "))
        cfc.gain_shares = int(input("Gain/(loss) on the disposal or revaluation/impairment of shares in other companies? "))
        cfc.gain_ip = int(input("Gain/(loss) on the disposal or revaluation/impairment of IP that was created or acquired before 1 April 2002? "))
        cfc.gain_cap_other = int(input("Any other profit/(loss) of a capital nature?\nNote that this does not include a profit/(loss) arising on a loan or on a trading balance? "))
        cfc.trust_inc = int(input("Income accrued to the trustees of a settlement in relation to which the Company is the beneficiary where that income was not taxed in the Company's territory? "))
        cfc.part_inc = int(input("The Company's share of any income which accrued to a partnership of which the Company is a partner where the income was not taxed in the Company's territory? "))
        if input("Did the Company have any loans or non-trading balances that were due from other group companies during the accounting period which did not carry an arm's length rate of interest? Type yes or no") == "yes":
            cfc.tp_adj = int(input("Please include the additional amount of interest income that would have arisen on those loans/non-trading balances during the accounting period if they had carried an arm's length rate of interest"))
        else:
            cfc.tp_adj = 0
    
        if cfc.lpe() == 'Missing values to determine non-trading income':
            cfc.div_inc = int(input("Dividends or other distributions received from other companies where a tax deduction is allowed in any territory in respect of the payment"))
            cfc.int_inc = int(input("Interest income accruing on loan assets and bank deposits?"))
            cfc.forex_gain_loans = int(input("Foreign exchange gain, both realised and unrealised, arising on loan assets and bank deposits?"))
            cfc.forex_gain_non_trade = int(input("Foreign exchange gain, both realised and unrealised, arising on loan liabilities used otherwise than to fund the Company's trading activities and/or property business?"))
            cfc.derivative_inc = int(input("Receipts from derivative contracts (e.g. swaps) which have been entered into otherwise than for trading purposes or otherwise than in relation to loan liabilities used to fund the Company's trading activities and/or property business? "))
            cfc.fin_lease_inc = int(input("Finance income from finance lease and/or hire purchase assets which are not used for the purposes of a trade? "))
            cfc.roy_inc = int(input("Royalty income from licensing of IP that does not form part of the Company's trading activities?"))
            cfc.non_trade_other = int(input("Any other income that is not considered to arise from the Company's trading activities?"))
            return cfc.lpe()
        else:
            return cfc.lpe()


In [48]:
com_exp_tst()

False

In [35]:
print(cfc.pbt, cfc.div_inc_no_ded, cfc.prof_prop, cfc.gain_tang_ass, cfc.gain_shares, cfc.gain_ip, cfc.gain_cap_other, cfc.trust_inc, cfc.part_inc, cfc.tp_adj)

499999 0 0 0 0 0 0 0 0 49999


In [39]:
cfc.lpe()

'Missing values to determine ap'

In [37]:
# create a sample company
cfc = Company(name = 'VGE',
              country_residence = 'Spain',
              currency = 'gbp',
                
              pbt = 5000000,
              prof_prop = 0,
              gain_tang_ass = 0,
              gain_shares = 0,
              gain_ip = 0,
              gain_cap_other = 0,
              trust_inc = 0,
              part_inc = 0,
              tp_adj = 10,

              gross_int_exp = 0,
              op_exp = -1000,
              op_exp_interco = 0,
              goods_costs = 0,
              
              exempt_inc = 0,
              low_tax_inc = 0,
              tax_paid = 0,
              notional_int = 0,
              ruling_inc = 0,
              tp_adj_asym = 0,
              
              div_inc = 49000,
              int_inc = 10,
              forex_gain_loans = 10,
              forex_gain_non_trade = 10,
              derivative_inc = 10,
              fin_lease_inc = 10,
              roy_inc = 10,
              non_trade_other = 10
              )

In [38]:
run_exemption_tests(cfc)

Simplified territory exemption: False 

Low profit exemption: Missing values to determine ap 

Low profit margin exemption: Missing values to determine ap 

Excluded territory exemption: False


Missing information to complete exemption tests


False